<div >
<img src="logo.jpg", width=150, ALIGN="left", border=20>
<center>
<h1>Final project</h1>
<br>This code was tested with <br>
Python 2.7.13 | Anaconda 4.3.1 (https://anaconda.org/)<br>
<i> Adapted for Chalab by Isabelle Guyon from original code of Balázs Kégl</i> <br>
<a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science (CDS)</a>
</center>
</div>

# Introduction

The aim of this competition is to predict at best the sales by some stores. This notebook helps the users in the preparation of submissions. 

In [1]:
# Keep this:
model_dir = 'sample_code_submission/'          
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
%matplotlib inline
%load_ext autoreload
%autoreload 2
import seaborn as sns; sns.set()

<div >
<h2> Step 1: Exploratory data analysis </h2>
Create a folder 'sample_data' and put the files in Data1 (available by github) inside it. This is the folder with the data and solutions for the training data. 
</div>

In [2]:
datadir = 'sample_data'              # Change this to the directory where you put the input data
dataname = 'store'
!ls $datadir*

sample_data:
store_feat.name   store_public.info  store_train.data	   store_valid.data
store_label.name  store_test.data    store_train.solution

sample_data_iris:
iris_feat.name	  iris_test.data      iris_train.solution
iris_label.name   iris_test.solution  iris_valid.data
iris_public.info  iris_train.data     iris_valid.solution


For convenience, we load the data as a "pandas" data frame, so we can use "pandas" and "seaborn" built in functions to explore the data. 

In [3]:
from data_io import read_as_df
data = read_as_df(datadir  + '/' + dataname)                # The data are loaded as a Pandas Data Frame
target_name = data.columns.values[-1]                       # The last column is the target

Reading sample_data/store_train from AutoML format
Number of examples = 712045
Number of features = 8
Number of classes = 1


In [30]:
data.head()

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,target
0,625,3,2013-11-06,641,1,1,0,0,Sales
1,293,2,2013-07-16,877,1,1,0,1,Sales
2,39,4,2014-01-23,561,1,1,0,0,Sales
3,676,4,2013-09-26,1584,1,1,0,0,Sales
4,709,3,2014-01-22,1477,1,1,0,0,Sales


In [7]:
data.describe() 

,Store,DayOfWeek,Customers,Open,Promo,SchoolHoliday
count,712045.000000,712045.000000,712045.000000,712045.000000,712045.000000,712045.000000
mean,558.314199,4.000265,633.399958,0.830174,0.381519,0.178543
std,321.898476,1.996580,464.360651,0.375480,0.485760,0.382970
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,405.000000,1.000000,0.000000,0.000000
50%,558.000000,4.000000,609.000000,1.000000,0.000000,0.000000
75%,837.000000,6.000000,838.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,5494.000000,1.000000,1.000000,1.000000


# Step 2: Building a predictive model


## Loading data with DataManager
We reload the data with the AutoML DataManager class because this is more convenient.

Note that we use `conv_to_num=False`, this means that categorical target values encoded in AutoML format with the 1-hot encoding are NOT converted to categorical labels (which is what scikit-learn assumes). Hence this conversion will need to be done before you call any scikit-learn model. Be aware that the variable 'date' is a string and the following is not going to work, unless the variable is transformed into a Float number, with a suitable transformation. Make this transformation before running the following code.

In [19]:
from data_manager import DataManager
#D = DataManager(dataname, datadir, replace_missing=True, conv_to_num=False)
D = DataManager(dataname, datadir, replace_missing=True, verbose=False)
print(D)

Info file found : /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_data/store_public.info
filename= sample_data/store_train.data
data_matrix [['625' '3' '2013-11-06' '641' '1' '1' '0' '0']
 ['293' '2' '2013-07-16' '877' '1' '1' '0' '1']
 ['39' '4' '2014-01-23' '561' '1' '1' '0' '0']]
after:  [['625' '16015.0' '641' '1' '1' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0'
  '0' '0']
 ['293' '15902.0' '877' '1' '1' '1' '1' '0' '0' '0' '0' '1' '0' '0' '0'
  '0' '0']
 ['39' '16093.0' '561' '1' '1' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0'
  '0' '0']]
filename= sample_data/store_train.solution
filename= sample_data/store_valid.data
data_matrix [['731' '1' '2014-02-24' '920' '1' '0' '0' '0']
 ['293' '4' '2013-01-03' '754' '1' '0' '0' '1']
 ['650' '1' '2013-12-30' '785' '1' '0' '0' '1']]
after:  [['731' '16125.0' '920' '1' '0' '0' '1' '0' '0' '0' '1' '0' '0' '0' '0'
  '0' '0']
 ['293' '15708.0' '754' '1' '0' '1' '1' '0' '0' '0' '0' '0' '0' '1' '0'
  '0' '0']
 ['650' '16069.0' '785' '1

In [48]:
from ingestion_program.data_io import data
data('sample_data/store_train.data', None, False)

before label:  [['625' '3' '16015.0' '641' '1' '1' '0' '0']
 ['293' '2' '15902.0' '877' '1' '1' '0' '1']
 ['39' '4' '16093.0' '561' '1' '1' '0' '0']]


AttributeError: 'numpy.ndarray' object has no attribute 'to_array'

## Training a predictive model
We provide an example of predictive model (for classification or regression) in the `sample_code_submission/` directory. It is a quite stupid model: it makes constant predictions. Replace it with your own model.

In [5]:
from model import model
# ??model 

Using TensorFlow backend.


<div >
<h3>Create the model</h3>
Create an instance of the model (run the constructor) 
</div>

In [6]:
M = model()   
trained_model_name = model_dir + dataname

<h3> Training:</h3>

In [7]:
X_train = D.data['X_train']
Y_train = D.data['Y_train']

In [8]:
X_train[:10]

array([[6.2500e+02, 1.6015e+04, 6.4100e+02, 1.0000e+00, 1.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [2.9300e+02, 1.5902e+04, 8.7700e+02, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [3.9000e+01, 1.6093e+04, 5.6100e+02, 1.0000e+00, 1.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [6.7600e+02, 1.5974e+04, 1.5840e+03, 1.0000e+00, 1.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.0900e+02, 1.6092e+04, 1.4770e+03, 1.0000e+

In [8]:
X_train = D.data['X_train']
Y_train = D.data['Y_train']

if not(M.is_trained):               # No need to train if model already trained
    M.fit(X_train, Y_train)  

/home/kihansi/miniconda3/envs/cert-big-data/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### Making predictions:

In [9]:
Y_hat_train = M.predict(D.data['X_train']) # Optional, not really needed to test on taining examples
Y_hat_valid = M.predict(D.data['X_valid'])
Y_hat_test = M.predict(D.data['X_test'])

## Scoring the results
### Load the challenge metric
<b>The metric chosen for your challenge</b> is identified in the "metric.txt" file found in the `scoring_function/` directory. We use here the `mse_metric` metric (an example of organizer-supplied metric found in `my_metric.py`), which computes the mean-square-error. 

In [10]:
with open(score_dir + 'metric.txt', 'r') as f:
    metric_name = f.readline().strip()
import libscores, my_metric
try:
    scoring_function = getattr(libscores, metric_name)
except:
    scoring_function = getattr(my_metric, metric_name)
print('Using scoring metric:', metric_name)
# ??scoring_function

Using scoring metric: mse_metric


### Training performance
The participants posess target values (labels) only for training examples. We compute with the mse metric the training score, which should be zero for perfect predictions. 

In [12]:
print ('Training score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_hat_train))
print ('Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_train))

Training score for the mse_metric metric = 135833.7550
Ideal score for the mse_metric metric = 0.0000


<div >
<H3> Cross-validation performance </H3>
The participants do not have access to the labels Y_valid and Y_test to self-assess their validation and test performances. But training performance is not a good prediction of validation or test performance. We suggest to use a cross-validation. Using cross-validation, the training data is split into multiple training/test folds, which allows participants to self-assess their model during development. <br>
</div>

In [13]:
from sklearn.model_selection import KFold
from numpy import zeros, mean
# 3-fold cross-validation
n = 3
kf = KFold(n_splits=n)
kf.get_n_splits(X_train)
i=0
scores = zeros(n)
for train_index, test_index in kf.split(X_train):
    Xtr, Xva = X_train[train_index], X_train[test_index]
    Ytr, Yva = Y_train[train_index], Y_train[test_index]
    M = model()
    M.fit(Xtr, Ytr)
    Yhat = M.predict(Xva)
    scores[i] = scoring_function(Yva, Yhat)
    print ('Fold', i+1, 'example metric = ', scores[i])
    i=i+1
print ('Average score = ', mean(scores))

/home/kihansi/miniconda3/envs/cert-big-data/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fold 1 example metric =  868824.6064312891


/home/kihansi/miniconda3/envs/cert-big-data/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fold 2 example metric =  881122.6468270642


/home/kihansi/miniconda3/envs/cert-big-data/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fold 3 example metric =  863367.9661238351
Average score =  871105.073127396


# Step 3: Making a submission

## Unit testing

A possibility to prepare a submission is the following one.  Modify the file <code>model.py</code> in the <code>sample_code_submission/</code> directory, then run this test to make sure everything works fine. The results of this program will be used on the server to test your submission. 

In [14]:
outdir = 'sample_result_submission'     

In [15]:
!python $problem_dir/ingestion.py $datadir $outdir $problem_dir $model_dir

Using input_dir: /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_data
Using output_dir: /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_result_submission
Using program_dir: /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/ingestion_program
Using submission_dir: /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_code_submission
Using TensorFlow backend.

========== Ingestion program version 6 ==========

************************************************
******** Processing dataset Store ********
************************************************
========= Reading and converting data ==========
Info file found : /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_data/store_public.info
========= Reading /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_data/store_feat.type
[+] Success in  0.00 sec
========= Reading /home/kihansi/Projects/FinalProject_CertBigData/starting_kit/sample_data/st

## Preparing the submission

To prepare the submission, you can run the command in the previous cell on the appropriate data (testing or validation data), and then zip the contents of `sample_result_submission/` (without the directory, and remember to remove the examples that are already there).

However, it is not necessary to run the previous code to prepare a submission. The site for the submission checks the predicted results against the true labels, through the scoring function. It accepts any zip file conatining files with extension '.predict'. It is important however that the name of the file matches that of the data. Example: to submit labels for store_train.data, the zip file has to contain a file named store_train.predict, with the same structure of the ones in `sample_result_submission/`
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>. After the submission you are given your score. If the result is close to zero, the prediction is good. 